# Sequence to Sequence Preprocessing in Pytorch

**|| Jonty Sinai ||** 28-04-2019

In this notebook I show how a **translation dataset** can be preprocessed into pairs of input and target sequences for training a sequence-to-sequence model with PyTorch. This dataset will then be used to test the implementation of two of the original sequence-to-sequence architectures introduced in [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) (Ilya Sutskever et al 2014) and [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)(Kyunghyun Cho et al 2014) respectively. 

The dataset which I will use is a simplified English-French translation dataset based on the [official PyTorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py). Whereas in my [ResNet implementation](https://github.com/JontySinai/artificial_neural_networks/blob/master/notebooks/resnet.ipynb) the goal was to explore modular composeability with PyTorch, the goal here is to explore sequence-to-sequence design patterns and engineering with PyTorch.

> **Note:** Compared to computer vision, feature selection and data preprocessing in natural language processing (NLP) requires more care and attention. There is no correct way of preprocessing text, although there are many incorrect ways. Choices must be made carefully and efficiently. As a result, a good portion of this notebook is spent preprocessing the dataset and converting it into the right format for sequence-to-sequence learning.

In [1]:
import os
import re
import random
import unicodedata
from io import open
from collections import defaultdict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

import numpy as np

HOME = os.environ['AI_HOME']
ROOT = os.path.join(HOME, 'artificial_neural_networks')
DATA = os.path.join(ROOT, 'data')
ENG_FR = os.path.join(DATA, 'english_french')

random.seed(1901)
np.random.seed(1901)
torch.manual_seed(1901)

## Data Preprocessing

English-French sentence pairs can be downloaded from the PyTorch translation tutorial [here](https://download.pytorch.org/tutorial/data.zip) (the original data comes from [https://tatoeba.org/eng/downloads](https://tatoeba.org/eng/downloads) and has been paired thanks to [https://www.manythings.org/anki/](https://www.manythings.org/anki/)). 

We will need to do the hardwork of preprocessing this data into tensors which we can feed into our model. We'll start by exploring the dataset and then understanding the I/O requirements of the model architecture. This will inform our choices for preprocessing. Since this is a sequence-to-sequence model, we'll need to be careful to ensure that pairs of input-output sequences are aligned during training and evaluation.

### Data Preview

The data is stored on one file with each line representing an English-French sentence pair. Each English and French sentence is tab separated.

In [2]:
data_path = os.path.join(ENG_FR, 'eng-fra.txt')

In [3]:
raw_pairs = open(data_path, encoding='utf-8').read().strip().split('\n')

for line in raw_pairs[:10]:
    print(line)

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Stop!	Ça suffit !
Stop!	Stop !
Stop!	Arrête-toi !


### Unicode Normalisation

As you can see there are some Unicode characters such as "À" and "Ç". Raw Unicode can be problematic for deep learning as seemingly identical characters on screen can have different representations on disk. For this reason we will need to _normalise_ Unicode characters so that representation is consistent. For more detail on why Unicode normalisation is important see this blogpost: [When "Zoë" !== "Zoë". Or why you need to normalize Unicode strings](https://withblue.ink/2019/03/11/why-you-need-to-normalize-unicode-strings.html) by [Alessandro Segala](https://withblue.ink/about.html). 

Unicode characters can be normalised by converting them into a so-called "canonical form" of which there are four. We will used **NFD** or [Unicode Normal Form Decomposition](https://unicode.org/reports/tr15/#Norm_Forms) to decompose characters with accents. For example "À" will be decomposed into "A" and "̖". We will then have to remove the accent using a filter on so-called _non-spacing mark_ characters. The solution is taken from the PyTorch tutorial (originally StackOverflow) below:

In [4]:
def unicode_normalisation(s):
    # ref: https://stackoverflow.com/a/518232/2809427
    return ''.join(
        char for char in unicodedata.normalize('NFD', s)  # "À" -->  "A" + "̖"
        if unicodedata.category(char) != 'Mn')  # remove "̖"s

In [5]:
for line in raw_pairs[:10]:
    print(unicode_normalisation(line))

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ca alors !
Fire!	Au feu !
Help!	A l'aide !
Jump.	Saute.
Stop!	Ca suffit !
Stop!	Stop !
Stop!	Arrete-toi !


We will then remove all non-letter characters such as numbers and punctuation, keeping only the end of sentence markers ".", "!", "?", which we will add a space in front of so we can treat them as separate tokens. As part of standard string normalisation for NLP we will convert all characters to lowercase and strip whitespace. 

>**Note:** as a result words like "aren't" will become "aren" "t" and will be treated as separate tokens.

In [6]:
def normalise_string(s):
    s = unicode_normalisation(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)  # add space in front of ".", "!" "?"
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)  # remove unwanted characets
    return s

In [7]:
for line in raw_pairs[:10]:
    print(normalise_string(line))

go . va !
run ! cours !
run ! courez !
wow ! ca alors !
fire ! au feu !
help ! a l aide !
jump . saute .
stop ! ca suffit !
stop ! stop !
stop ! arrete toi !


### Tokenisation

The next step is to tokenise each sentence. Tokenisation is useful because it reduces sentences (which occur uniquely in the dataset) into smaller units. We can then represent the _feature space_ of each language in terms of these tokens. This feature space is orders of magnitude smaller than the sentence space (cardinality ~ dataset) and allows us to represent sentences in terms of shared features. Finally we can treat each _timestep_ in each sentence as a token. 

> Thus we can represent each sentence as a **sequence of tokens**.

One important aspect of tokenisation is that we can _index each unique token_ for each language to form what is known as **vocabulary**, which is _1-to-1_ mapping between indices and tokens.

An example of a vocabulary mapping, containing two special tokens `<SOS>` and `<EOS>` start/end of sequence markers, is as follows:

<img src="assets/word-encoding.png">

In [8]:
def tokenise(sentence):
    return sentence.split(' ')

def tokenise_pair(pair):
    return [tokenise(normalise_string(s)) for s in pair.split('\t')]

In [9]:
for line in raw_pairs[:5]:
    print(tokenise_pair(line)[0], '\t', tokenise_pair(line)[1])

['go', '.'] 	 ['va', '!']
['run', '!'] 	 ['cours', '!']
['run', '!'] 	 ['courez', '!']
['wow', '!'] 	 ['ca', 'alors', '!']
['fire', '!'] 	 ['au', 'feu', '!']


Each sentence in each pair is now a sequence of tokens.

> **Note:** Those familiar with standard NLP preprocessing will know that the tokenisation process is typically followed by the removal of stopwords (such as "the", "a"), pronouns (such as "they") and lemmatisation ("going" --> "go"). For neural machine translation we do want to keep stopwords, pronouns and tense as these are important for accurate translation.

#### Onehot-Vectors

Typicall we convert each token into a onehot-vector of dimensionality $\mathbb{R}^{\lvert V \rvert}$ where $\lvert V \rvert$ is the vocabulary size and each direction is a unique token.

In the following example, taken from Andrew Ng's Deep Learning Course, an example English vocabulary could contain 10 000 unique tokens, including a special `"<UNK>"` token to represent unknown tokens. If **"queen"** has index **7157** then it will have a $1$ at the 7157<sup>th</sup> direction in $\mathbb{R}^{\lvert V \rvert}$ and $0$ everywhere else.

<img src="assets/onehot-vectors.png" width="600">
<br/>

#### Continuous Vector Embeddings

The problem with onehot vector representations is that they encode information inefficiently. In a vector of length 10,000, 9,999 directions contain zero information content and 1 direction contains 100% of the information content! A well studied way of encoding information more efficiently is to _compress_ the _sparse_ onehot vector into a _dense_ or _continuous_ vector representation with smaller dimensionality. We lose the meaning of each direction corresponding to a unique vector but we gain a memory cost saving. 

> Even more than that, there is a whole line of research studying continuous vector embeddings where the goal is to learn meaningful directions in an unsupervised way. The most famous of which are [word2vec](https://arxiv.org/abs/1301.3781), [fastText](https://arxiv.org/abs/1607.01759) and [GloVe](https://nlp.stanford.edu/projects/glove/). The encoding efficiency of these embeddings have been shown to improve downstream tasks such as sequence-to-sequence modelling. 

Using such pre-trained embeddings, words such as **"queen"** can now be decomposed into a smaller, continuous vector where certain directions \*may have certain encoded meanings. 

<img src="assets/embedding-vectors.png" width="600">
<br/>

### Sequence Encoding

Since we will typically pass onehot vectors through an initial **embedding layer (transformation)**, we do not need to represent each token as a onehot vector in PyTorch. This is because under the hood, PyTorch will precompute the embeddings for each unique token and use a lookup-table to map each token to its embedding vector. Thus we can instead just use the token index at each timestep so that the word **"queen"** occuring at time $t$ will simply be represented by **7157** at time $t$.

>Thus by indexing tokens uniquely for each language we can represent each sentence as a **sequence of indices**.

We will manage tokenisation, indexing and sequence conversion using a **Vocabulary** class. This class will be responsible for:

- Mapping tokens to indices and vice-versa.
    - This will include the three special tokens mentioned earlier, namely `<SOS>`, `<EOS>` and `<UNK>`.
- Adding tokens to the vocabulary.
- Keeping track of vocabulary size.

We will then use this class to convert **sequences of tokens** into **sequences of indices**. 

In [10]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3

class Vocab:
    
    def __init__(self, name):
        self.name = name
        self.token2index = defaultdict(lambda: UNK_TOKEN,  # return UNK_TOKEN if unknown token encountered
                                       {"<pad>": PAD_TOKEN, "<sos>": SOS_TOKEN, "<eos>": EOS_TOKEN})  # init key-vals
        self.index2token = {PAD_TOKEN: "<pad>", SOS_TOKEN: "<sos>", EOS_TOKEN: "<eos>", UNK_TOKEN: "<unk>"}
        self.size = len(self.index2token)  # initially 4
        
    def add_token(self, token):
        if token not in self.token2index:
            self.token2index[token] = self.size  # eg first new word has index 4
            self.index2token[self.size] = token  # 1-to-1 mapping
            self.size += 1
            
    def __len__(self):
        return self.size
    
    def __str__(self):
        return str(self.token2index)
    

### Test: Vocab

Let's now test the vocabulary class on the first 10 pairs, showing how it will be used to create sequences of indices from sequences of tokens.

In [11]:
en_token_sequences = []
fr_token_sequences = []
for line in raw_pairs[:10]:
    pair = tokenise_pair(line)
    en_token_sequences.append(pair[0])
    fr_token_sequences.append(pair[1])
    
print(en_token_sequences)
print('\n', fr_token_sequences)

[['go', '.'], ['run', '!'], ['run', '!'], ['wow', '!'], ['fire', '!'], ['help', '!'], ['jump', '.'], ['stop', '!'], ['stop', '!'], ['stop', '!']]

 [['va', '!'], ['cours', '!'], ['courez', '!'], ['ca', 'alors', '!'], ['au', 'feu', '!'], ['a', 'l', 'aide', '!'], ['saute', '.'], ['ca', 'suffit', '!'], ['stop', '!'], ['arrete', 'toi', '!']]


In [12]:
en_vocab = Vocab("en")
print(en_vocab)

defaultdict(<function Vocab.__init__.<locals>.<lambda> at 0x11cfc1730>, {'<pad>': 0, '<sos>': 1, '<eos>': 2})


In [13]:
for sentence in en_token_sequences:
    for token in sentence: en_vocab.add_token(token)
        
print(en_vocab)

defaultdict(<function Vocab.__init__.<locals>.<lambda> at 0x11cfc1730>, {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'go': 4, '.': 5, 'run': 6, '!': 7, 'wow': 8, 'fire': 9, 'help': 10, 'jump': 11, 'stop': 12})


In [14]:
en_index_sequences = []
for sentence in en_token_sequences[:10]:
    sequence = [en_vocab.token2index[t] for t in sentence]
    en_index_sequences.append(sequence)

print(en_index_sequences)

[[4, 5], [6, 7], [6, 7], [8, 7], [9, 7], [10, 7], [11, 5], [12, 7], [12, 7], [12, 7]]


In [15]:
for token_seq, idx_seq in zip(en_token_sequences, en_index_sequences):
    print(token_seq, ' : ', idx_seq)

['go', '.']  :  [4, 5]
['run', '!']  :  [6, 7]
['run', '!']  :  [6, 7]
['wow', '!']  :  [8, 7]
['fire', '!']  :  [9, 7]
['help', '!']  :  [10, 7]
['jump', '.']  :  [11, 5]
['stop', '!']  :  [12, 7]
['stop', '!']  :  [12, 7]
['stop', '!']  :  [12, 7]


And we can do the same for French

In [16]:
fr_vocab = Vocab("fr")

for sentence in fr_token_sequences:
    for token in sentence: fr_vocab.add_token(token)

fr_index_sequences = []
for sentence in fr_token_sequences[:10]:
    sequence = [fr_vocab.token2index[t] for t in sentence]
    fr_index_sequences.append(sequence)
    
for token_seq, idx_seq in zip(fr_token_sequences, fr_index_sequences):
    print(token_seq, ' : ', idx_seq)

['va', '!']  :  [4, 5]
['cours', '!']  :  [6, 5]
['courez', '!']  :  [7, 5]
['ca', 'alors', '!']  :  [8, 9, 5]
['au', 'feu', '!']  :  [10, 11, 5]
['a', 'l', 'aide', '!']  :  [12, 13, 14, 5]
['saute', '.']  :  [15, 16]
['ca', 'suffit', '!']  :  [8, 17, 5]
['stop', '!']  :  [18, 5]
['arrete', 'toi', '!']  :  [19, 20, 5]


In [17]:
print(fr_vocab)

defaultdict(<function Vocab.__init__.<locals>.<lambda> at 0x11cfc19d8>, {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'va': 4, '!': 5, 'cours': 6, 'courez': 7, 'ca': 8, 'alors': 9, 'au': 10, 'feu': 11, 'a': 12, 'l': 13, 'aide': 14, 'saute': 15, '.': 16, 'suffit': 17, 'stop': 18, 'arrete': 19, 'toi': 20})


### Tensor Sequences in PyTorch

The next step is to convert the sequence of indices into tensors for training and prediction using PyTorch. At it's most basic we can convert an array to a tensor as follows:

In [18]:
en_tensor_samples = []
for idx_seq in en_index_sequences:
    tensor_seq = torch.tensor(idx_seq)
    en_tensor_samples.append(tensor_seq)
    print(idx_seq, ' : ', tensor_seq)

[4, 5]  :  tensor([4, 5])
[6, 7]  :  tensor([6, 7])
[6, 7]  :  tensor([6, 7])
[8, 7]  :  tensor([8, 7])
[9, 7]  :  tensor([9, 7])
[10, 7]  :  tensor([10,  7])
[11, 5]  :  tensor([11,  5])
[12, 7]  :  tensor([12,  7])
[12, 7]  :  tensor([12,  7])
[12, 7]  :  tensor([12,  7])


In [19]:
fr_tensor_samples = []
for idx_seq in fr_index_sequences:
    tensor_seq = torch.tensor(idx_seq)
    fr_tensor_samples.append(tensor_seq)
    print(idx_seq, ' : ', tensor_seq)

[4, 5]  :  tensor([4, 5])
[6, 5]  :  tensor([6, 5])
[7, 5]  :  tensor([7, 5])
[8, 9, 5]  :  tensor([8, 9, 5])
[10, 11, 5]  :  tensor([10, 11,  5])
[12, 13, 14, 5]  :  tensor([12, 13, 14,  5])
[15, 16]  :  tensor([15, 16])
[8, 17, 5]  :  tensor([ 8, 17,  5])
[18, 5]  :  tensor([18,  5])
[19, 20, 5]  :  tensor([19, 20,  5])


Using the basic design pattern above, we now have everything we need to construct a PyTorch dataset. However, before we do so, it will be helpful to look at the generic sequence-to-sequence architecture so that we can get a better understanding of the I/O requirements of the model.

## Sequence-to-Sequence Model Architecture

A **sequence-to-sequence** architecture is a generic type of deep learning architecture consisting of two main components:

- An **encoder** which takes in an **input sequence** and encodes it into some **latent representation**, sometimes called the **context vector**.
- A **decoder** which takes in the latent representation and produces an **output sequence**.
    - The decoder may also take in an input sequence of its own. During training this is typically the target sequence. During prediction, this is the predicted sequence itself.
    - During training the decoder can be thought of as a **generative model** which tries to produce the target sequence, given the context vector.

For example, in the French $\rightarrow$ English translation task, a sequence-to-sequence model will look as follows

<img src="assets/seq2seq.png">

Here the output sequence is fed back into the decoder for prediction. 

One thing to note is that for the encoder we can process the entire sequence at once since we are only interested in the final hidden state of the encoder, however since the decoder is a generative model it does not make sense to process the entire sequence as a batch since the decoder at time $t$ will depend on its output at time $t-1$. Thus the decoder will typically be implemented with a loop in the forward pass.

## Sequence Batching

Typically we would like to process the data in minibatches. At first this might seem confusing since the decoder needs to loop through each sequence one element at a time, however the PyTorch implementations of recurrent modules support batch processing provided that its input has shape `sequence_length x batch_size x input_dimension` where _input dimension_ will be the _embedding dimension_. This way we can loop over each sequence in the batch one element at a time by indexing over the first dimension. 

>Since we're using embeddings, the dimensionality of the batch will be transformed as:
<br/><br/>`sequence_length x batch_size` $\rightarrow$ embedding $\rightarrow$ `sequence_length x batch_size x embedding_dimension` $\rightarrow$ recurrent unit

We'll see later that _sequence length_ will have to be the _max sequence length_ of the batch.

Let's see how this works by considering the test tensors we created earlier for the English samples.

In [20]:
en_tensor_batch = torch.stack(en_tensor_samples)

print(en_tensor_batch)
print('\n', en_tensor_batch.size())
print('\n', 'sequence at batch index 0: ', en_tensor_batch[0])

tensor([[ 4,  5],
        [ 6,  7],
        [ 6,  7],
        [ 8,  7],
        [ 9,  7],
        [10,  7],
        [11,  5],
        [12,  7],
        [12,  7],
        [12,  7]])

 torch.Size([10, 2])

 sequence at batch index 0:  tensor([4, 5])


As you can see, we will need to reshape this batch so that indexing over the first dimension corresponds to time (indexing each sequence at each time step) and not to sample (indexing over each batch).

In [21]:
en_tensor_batch = en_tensor_batch.transpose(1, 0)

print(en_tensor_batch)
print('\n', en_tensor_batch.size())
print('\n', 'sequence elements at time 0: ', en_tensor_batch[0])

tensor([[ 4,  6,  6,  8,  9, 10, 11, 12, 12, 12],
        [ 5,  7,  7,  7,  7,  7,  5,  7,  7,  7]])

 torch.Size([2, 10])

 sequence elements at time 0:  tensor([ 4,  6,  6,  8,  9, 10, 11, 12, 12, 12])


Now indexing in the first dimension (by $t$) gives us a subset of the batch where the $i^{th}$ element corresponds to the $t^{th}$ timestep of the $i^{th}$ sequence in the batch. This is the dimensionality which we will require from batching during the dataloading process.

### Batch Sequence Padding

You may have noticed that all the tensors in the English samples conveniently have the same length whereas with the French samples the tensors have different lengths. This will be problematic for indexing over time as we will either get index-out-of-bounds errors for shorter sequences or we will fail to index longer sequences fully. 

The way we deal with this is sequence padding to the right to the length of the longest sequence in the batch. Earlier you may have noticed that the first index $0$ in each vocabulary was reserved for padding. By padding with $0$ no information will be lost or gained when transforming padded sequences through each recurrent unit.

>**Note:** It's worth mentioning that in cases where there are very long sequences in the training data, then a maximum sequence length parameter is usually set to control memory during processing. Sequences exceding the length limit can then be rolled over onto the next batch.
For example if the sequence length limit is $4$ but we have a sequence, `[4, 5, 9, 12, 31]` of length $5$ then it will be rolled over the batch as 
```python
tensor([[ 4,  5,  9, 12],
        [31,  0,  0,  0]])
```
which is transposed to
```python
tensor([[ 4, 31],
        [ 5,  0],
        [ 9,  0],
        [12,  0]]),
```


Let's see how this padding will work in practice using the `nn.utils.rnn.pad_sequence` utility. First notice what happens when we don't pad:

In [22]:
fr_tensor_batch = torch.stack(fr_tensor_samples)

print(fr_tensor_batch)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 2 and 3 in dimension 1 at /Users/soumith/mc3build/conda-bld/pytorch_1549597882250/work/aten/src/TH/generic/THTensorMoreMath.cpp:1307

In [23]:
fr_tensors_padded = nn.utils.rnn.pad_sequence(fr_tensor_samples)

print(fr_tensors_padded, '\n')

for t in range(4):
    print(f'sequence elements at time {t}: {fr_tensors_padded[t]}')

tensor([[ 4,  6,  7,  8, 10, 12, 15,  8, 18, 19],
        [ 5,  5,  5,  9, 11, 13, 16, 17,  5, 20],
        [ 0,  0,  0,  5,  5, 14,  0,  5,  0,  5],
        [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0]]) 

sequence elements at time 0: tensor([ 4,  6,  7,  8, 10, 12, 15,  8, 18, 19])
sequence elements at time 1: tensor([ 5,  5,  5,  9, 11, 13, 16, 17,  5, 20])
sequence elements at time 2: tensor([ 0,  0,  0,  5,  5, 14,  0,  5,  0,  5])
sequence elements at time 3: tensor([0, 0, 0, 0, 0, 5, 0, 0, 0, 0])


Conveniently PyTorch's RNN padding utility also transposes the sequences into the right dimensionality for PyTorch's reccurent modules.

### Batch Embeddings

Before we pass the batched sequences through the recurrent cell, we need to pass them through an embedding layer. In PyTorch we can instruct the embedding layer to preserve the padding index, so that sequence length information is not lost during the embedding transformation.

In [24]:
fr_embedding = nn.Embedding(num_embeddings=len(fr_vocab), embedding_dim=3, padding_idx=0)

print(fr_embedding)

Embedding(21, 3, padding_idx=0)


It is quite interesting to see what the embedding look like. The dimensionality for sequence data is quite confusing but as we will see, by extracting timesteps from the embedded batch, we can try and understand its shape.

In [25]:
fr_embedded_tensors = fr_embedding(fr_tensors_padded)

print(fr_embedded_tensors)

tensor([[[ 0.3826,  0.2643, -0.4163],
         [-1.1180,  0.2400, -0.6511],
         [-0.1632, -0.2048,  0.2239],
         [-0.0987,  1.3309,  2.1294],
         [-0.0869, -1.1208,  0.2758],
         [ 0.3475,  2.1952, -0.4877],
         [ 0.4658, -0.7564,  0.6001],
         [-0.0987,  1.3309,  2.1294],
         [-0.9418, -0.6638,  0.9538],
         [ 0.2485,  0.1805, -1.0135]],

        [[ 0.9746,  0.4226, -0.5363],
         [ 0.9746,  0.4226, -0.5363],
         [ 0.9746,  0.4226, -0.5363],
         [-0.9805,  0.3202,  1.0725],
         [-0.2719, -0.2972,  0.6684],
         [-1.2755,  0.6203, -0.8780],
         [ 0.2191,  1.0461,  0.8399],
         [-0.5260,  0.9915,  0.1009],
         [ 0.9746,  0.4226, -0.5363],
         [ 1.0638,  0.3519, -1.0559]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.9746,  0.4226, -0.5363],
         [ 0.9746,  0.4226, -0.5363],
         [-0.1795,  0.4190,  1.5105],
        

Let's interpret how the embedding has transformed the padded batch:

In [26]:
print('Before embedding: ', fr_tensors_padded.size())
print('After embedding: ', fr_embedded_tensors.size())

Before embedding:  torch.Size([4, 10])
After embedding:  torch.Size([4, 10, 3])


Each sequence was previously represented columnwise in the batch. Now each sequence is a 2-dimensional slice along the first and third dimension of the batch (which we can think of as a 3d volume of "slices" where each slice is a sequence):

In [27]:
print('Before embedding:\n')
print(fr_tensors_padded[:, 0])
print('\nAfter embedding:\n')
print(fr_embedded_tensors[:, 0, :])

Before embedding:

tensor([4, 5, 0, 0])

After embedding:

tensor([[ 0.3826,  0.2643, -0.4163],
        [ 0.9746,  0.4226, -0.5363],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward>)


Notice how the padding has been preserved. 

Also notice that **time** is still indexed along the first dimension of the tensor (going downwards). This will still hold true for the entire batch.

In [28]:
print(fr_embedded_tensors[0])

tensor([[ 0.3826,  0.2643, -0.4163],
        [-1.1180,  0.2400, -0.6511],
        [-0.1632, -0.2048,  0.2239],
        [-0.0987,  1.3309,  2.1294],
        [-0.0869, -1.1208,  0.2758],
        [ 0.3475,  2.1952, -0.4877],
        [ 0.4658, -0.7564,  0.6001],
        [-0.0987,  1.3309,  2.1294],
        [-0.9418, -0.6638,  0.9538],
        [ 0.2485,  0.1805, -1.0135]], grad_fn=<SelectBackward>)


Here each row of this slice corresponds to the _first time step_ of each sequence. See how the 3-dimensional tensor embedding for index $4$, <br/>`[0.3826,  0.2643, -0.4163]`, matches the first time step of the first sequence.

### Batched Forward Propagation

The embedded batch now has the right dimensionality for processing through a recurrent cell. 

In [29]:
print(fr_embedded_tensors.size())

torch.Size([4, 10, 3])


In [30]:
rnn = nn.RNN(input_size=3,  # embedding_dimension = 3
             hidden_size=4,
             num_layers=1,
             bidirectional=False) # num_directions = 1

h_0 = torch.randn(1, 10, 4)  # num_layers*num_directions x batch_size x hidden_size

In [31]:
output, h_n = rnn(fr_embedded_tensors, h_0)  # output corresponds to final time step: t = n = max_seq_len = 4

print(output.size())

torch.Size([4, 10, 4])


If we wanted we could also process the batch sequentially through time as we will in the decoder:

In [32]:
outputs = []
for t in range(4):
    embeddings_at_time_t = fr_embedded_tensors[t].view(1, 10, 3)  # 1 x batch_size x embedding_dimension
    output_t, h_t = rnn(embeddings_at_time_t, h_0)
    outputs.append(output_t)
    
    print(output_t.size())

torch.Size([1, 10, 4])
torch.Size([1, 10, 4])
torch.Size([1, 10, 4])
torch.Size([1, 10, 4])


## PyTorch Sequence Dataset

Now that we have an understanding of the correct tensor representation of the dataset, we can now construct a PyTorch `Dataset` class which will convert the data into tensors with the correct shape. We will then pass this dataset to a PyTorch `Dataloader` which will handle batching. It is through the dataloader class that the model will interface with the dataset.

When interfacing with sequence models we will want to focus as much time on the modelling choices so having gone through this whole notebook to implement a preprocessing strategy we'll ask this dataset class to not just handle tensor conversion but the whole data preprocessing from file to tensor representation. 

To recap this means that the dataset class will:

- read sentence pairs from a file
- normalise each sentence in each pair
- tokenise each pair
- build a vocabulary from the tokenised pairs for each language
- convert every pair into sequences of indices for each language
- convert every pair into tensors

During training and evaluation all we will ask of the dataloader class is to simply

- batch pairs of tensor sequences
- pad the input and target pairs respectively

Note that in the dataloader class padding will be applied locally to each batch using a special collate function. This is because the longest sequence length can differ from batch to batch, although it will be limited by `max_seq_length` throughout.

> **Note:** For this implementation we will use a special filter to limit the size of the dataset to reduce complexity. This will make it easier and faster to test implementations on limit resources. It will also mean that we won't encounter the sequence rollover problem when sequence length exceeds the maximum.

In [33]:
class TranslationDataset(Dataset):
    
    def __init__(self, path, lang1_name, lang2_name, max_seq_length, filter_pairs=False, reverse=False):
        """PyTorch dataset for translation data stored as tab separated sentence pairs, with each
        pair on one line of the file. The input language - lang1 - is the first sentence and the
        output language - lang2 - is the second sentence by default.
            
                  line:  'sentence1_lang1 \t sentence1_lang2'
        
            path: Path to text file containing tab separated sentence pairs on each line.                    
            lang1_name: Name to use for lang1 vocabulary.
            lang2_name: Name to use for lang2 vocabulary.
            max_seq_length: Maximum length of each sequence. If sequence 
                exceeds the max length it will be split into multiple 
                sequences.
            filter_pairs: If true will apply a special filter to simplify the
                dataset for faster experimentation.
            reverse: If true, will treat lang2 as the input and lang1 as the target
        
        TODO: implement sequence rollover for long sequences.
        
        """
        self.max_seq_length = max_seq_length
        self.reverse = reverse
        
        raw_pairs = self._read_pairs(path, filter_pairs)

        self.tokenised_pairs = [self._tokenise_pair(pair) for pair in raw_pairs]
        if self.reverse:
            self.tokenised_pairs = [list(reversed(pair)) for pair in self.tokenised_pairs]
        self.input_vocab, self.output_vocab = self._init_vocabs(lang1_name, lang2_name)
        
    def __getitem__(self, index):
        pair = self.tokenised_pairs[index]
        
        input_sequence = self.tokens2sequence(pair[0], self.input_vocab)
        target_sequence = self.tokens2sequence(pair[1], self.output_vocab, append_EOS=True)
        
        return input_sequence, target_sequence
        
    def tokens2sequence(self, tokens, vocab, append_EOS=False):
        sequence = [vocab.token2index[t] for t in tokens]
        if append_EOS:
            sequence.append(EOS_TOKEN)
        return torch.tensor(sequence)
    
    def _read_pairs(self, path, filter_pairs):
        raw_pairs = open(data_path, encoding='utf-8').read().strip().split('\n')
        print(f'Total number of sentence pairs: {len(raw_pairs)}')
        
        if filter_pairs:
            raw_pairs = [pair for pair in raw_pairs if self._keep(pair)]
            print(f'Total number of pairs after filtering: {len(raw_pairs)}')
            
        return raw_pairs
        
    def _init_vocabs(self, lang1_name, lang2_name):
        
        if self.reverse:
            input_vocab = Vocab(lang2_name)
            output_vocab = Vocab(lang1_name)
        else:
            input_vocab = Vocab(lang1_name)
            output_vocab = Vocab(lang2_name)
            
        for pair in self.tokenised_pairs:
            for token in pair[0]:
                input_vocab.add_token(token)
            for token in pair[1]:
                output_vocab.add_token(token)
                
        print('Total number of tokens in each vocabulary:')
        print(f'\tInput: {input_vocab.name}: {len(input_vocab)}')
        print(f'\tOutput: {output_vocab.name}: {len(output_vocab)}')
            
        return input_vocab, output_vocab
    
    def _tokenise_pair(self, pair):
        return [self._tokenise(self._normalise_string(s)) for s in pair.split('\t')]
    
    def _tokenise(self, sentence):
        #TODO: add sequence rollover here
        return sentence.split(' ')
    
    def _normalise_string(self, s):
        s = self._unicode_normalisation(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)  # add space in front of ".", "!" "?"
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)  # remove unwanted characets
        return s
        
    def _unicode_normalisation(self, s):
        # ref: https://stackoverflow.com/a/518232/2809427
        return ''.join(
            char for char in unicodedata.normalize('NFD', s)  # "À" -->  "A" + "̖"
            if unicodedata.category(char) != 'Mn')  # remove "̖"s

    def _keep(self, pair):
        """Special filter for reducing the size and complexity of the dataset. 

        Retains only pairs where the English sentence is of the form 

            "He is ... ", "She is ... " etc

        And filters pairs which are longer than max_seq_length. Note that this
        means that we will not encounter sequence rollover issues when batching.

        """
        en_sent_prefixes = ("I am", "I'm",
                             "He is", "He's",
                             "She is", "She's",
                             "You are", "You're",
                             "We are", "We're ",
                             "They are", "They're")

        split_pair = pair.split('\t')

        en_sentence = split_pair[0]
        fr_sentence = split_pair[1]

        condition =  len(fr_sentence.split(' ')) < self.max_seq_length and \
                     len(en_sentence.split(' ')) < self.max_seq_length and en_sentence.startswith(en_sent_prefixes)

        return condition
    
    def __len__(self):
        return len(self.tokenised_pairs)
        
        

In [34]:
en_fr_dataset = TranslationDataset(data_path, 'en', 'fr', 10, filter_pairs=True)

Total number of sentence pairs: 135842
Total number of pairs after filtering: 11893
Total number of tokens in each vocabulary:
	Input: en: 3058
	Output: fr: 4715


If we wanted to translate from French $\rightarrow$ English instead, then the `reverse` toggle will reverse the order of the tab separated pairs so that the French sentences are first the English sentences second. 

In [35]:
rev_en_fr_dataset = TranslationDataset(data_path, 'en', 'fr', 10, filter_pairs=True, reverse=True)

Total number of sentence pairs: 135842
Total number of pairs after filtering: 11893
Total number of tokens in each vocabulary:
	Input: fr: 4715
	Output: en: 3058


PyTorch's Dataloader class only needs to be able to index the dataset to collate batches. We can see how this works below:

In [36]:
en_fr_dataset[0]  # recall EOS_TOKEN = 2

(tensor([4, 5, 6]), tensor([4, 5, 6, 7, 2]))

In [37]:
en_fr_dataset[10]

(tensor([ 4,  5, 14,  6]), tensor([ 8, 13, 23,  7,  2]))

We can index the reversed dataset to see how the input and output pairs have been swapped.

In [38]:
rev_en_fr_dataset[0]

(tensor([4, 5, 6, 7]), tensor([4, 5, 6, 2]))

In [39]:
rev_en_fr_dataset[10]

(tensor([ 8, 13, 23,  7]), tensor([ 4,  5, 14,  6,  2]))

Notice how the EOS token is correctly append to only the output sequences.

## PyTorch Dataloader Batching

Finally let's see how we will interface with the dataset during training and evaluation using a PyTorch `Datalaoder`. Because PyTorch's Dataloader default collate function stacks tensors to form a batch, we will be unable to use the default collate function for sequences of variable lengths. So we will write our own collate function to unzip a list of tuples of input, target sequences, pad them seperately and then rezip the tuple as the padded batch.

PyTorch recurrent modules support a special type of sequence batch called a `PackedSequence` which can be used to calculate gradients on variable length sequences _while ignoring the padded values_. Unfortunately the embedding module does not support packed sequences so we will still have to pad each batch for the embedding, then use the `nn.utils.rnn.pack_padded_sequence` utility to pack the output of the embedding layer, which as we saw can be set to preserve padding.

To use the `pack_padded_sequence` utility we will have to sort the input tensors by their length and pass a tensor of lengths to pack the batch correctly. Unfortunately this means that the correspondence between the input and target sequences in each batch will be lost. This means that we will have to keep track of a _reverse sort_ to restore the original ordering of the encoder outputs before passing the hidden state to the decoder. 

> **Note:** It will not be necessary to pack the target sequences since the decoder is an autoregressive model which relies on its output at the previous time step. Thus we will only face the complexity of reordering the input sequences to restore within-batch order.

In [95]:
batch = [torch.tensor([1, 2]), torch.tensor([3, 4]), torch.tensor([5, 6, 7, 8, 9]), torch.tensor([10,11,12,13])]
print(batch)

[tensor([1, 2]), tensor([3, 4]), tensor([5, 6, 7, 8, 9]), tensor([10, 11, 12, 13])]


In [96]:
lengths = torch.tensor([len(t) for t in batch], dtype=torch.long)
print(lengths)

tensor([2, 2, 5, 4])


In [97]:
sorted_lengths, sort_idx = lengths.sort(descending=True)

max_length = sorted_lengths[0].item()

print(sorted_lengths)
print(sort_idx)

tensor([5, 4, 2, 2])
tensor([2, 3, 0, 1])


In [98]:
padded_batch = nn.utils.rnn.pad_sequence(batch)
print(padded_batch)

tensor([[ 1,  3,  5, 10],
        [ 2,  4,  6, 11],
        [ 0,  0,  7, 12],
        [ 0,  0,  8, 13],
        [ 0,  0,  9,  0]])


In [99]:
sort_idx.repeat(max_length, 1)

tensor([[2, 3, 0, 1],
        [2, 3, 0, 1],
        [2, 3, 0, 1],
        [2, 3, 0, 1],
        [2, 3, 0, 1]])

In [100]:
sorted_padded_batch = padded_batch.gather(dim=1, index=sort_idx.repeat(max_length, 1))  # max length
print(sorted_padded_batch)

tensor([[ 5, 10,  1,  3],
        [ 6, 11,  2,  4],
        [ 7, 12,  0,  0],
        [ 8, 13,  0,  0],
        [ 9,  0,  0,  0]])


In [101]:
packed_batch = nn.utils.rnn.pack_padded_sequence(sorted_padded_batch, sorted_lengths)

In [102]:
unpacked_batch, unpacked_lengths = nn.utils.rnn.pad_packed_sequence(packed_batch)

In [103]:
print(unpacked_batch)

tensor([[ 5, 10,  1,  3],
        [ 6, 11,  2,  4],
        [ 7, 12,  0,  0],
        [ 8, 13,  0,  0],
        [ 9,  0,  0,  0]])


In [104]:
print(unpacked_lengths)

tensor([5, 4, 2, 2])


In [105]:
unpacked_batch.gather(dim=1, index=sort_idx.repeat(max_length, 1))

tensor([[ 1,  3,  5, 10],
        [ 2,  4,  6, 11],
        [ 0,  0,  7, 12],
        [ 0,  0,  8, 13],
        [ 0,  0,  9,  0]])

In [165]:
def pad_and_collate(batch: list):
    input_batch, target_batch = zip(*batch)
    # we need to sort the batch in descending order of input lengths for the packing utility later
    input_lengths = torch.tensor([len(t) for t in input_batch], dtype=torch.long)
    sorted_lengths, sort_idx = input_lengths.sort(descending=True)
    # later we will need a reverse map of the sort idx to unsort the input batch
    _, original_idx = sort_idx.sort()
    max_length = sorted_lengths[0].item()
    # now pad
    input_padded = nn.utils.rnn.pad_sequence(input_batch)
    target_padded = nn.utils.rnn.pad_sequence(target_batch)
    # now sort by the input batch by length
    input_sorted = input_padded.gather(dim=1, index=sort_idx.repeat(max_length, 1))
    
    return input_sorted, input_padded, target_padded, sorted_lengths, original_idx
    

To see how the collate function works, let's construct a test batch of two sequence pairs and examine. Notice that each pair is a tuple so that at its most basic, the batch is a list of tuples.

In [166]:
test_batch = [en_fr_dataset[0], en_fr_dataset[1]]
test_batch

[(tensor([4, 5, 6]), tensor([4, 5, 6, 7, 2])),
 (tensor([4, 5, 7, 6]), tensor([ 8,  9, 10,  7,  2]))]

In [130]:
test_input_batch, test_target_batch, input_lengths, _ = pad_and_collate(test_batch)

Now we will see how the input sequence of each pair (tuple) has been extracted and padded independently of the target sequence of each pair. 

In [131]:
test_input_batch

tensor([[4, 4],
        [5, 5],
        [7, 6],
        [6, 0]])

And here is the target batch from our test batch. Notice how the input and target pairs are perfectly aligned along the batch dimension.

In [132]:
test_target_batch

tensor([[ 4,  8],
        [ 5,  9],
        [ 6, 10],
        [ 7,  7],
        [ 2,  2]])

In [167]:
en_fr_dataloader = DataLoader(en_fr_dataset, 
                              batch_size=10, 
                              num_workers=0, 
                              collate_fn=pad_and_collate)

In [168]:
en_fr_data_iterator = iter(en_fr_dataloader)
first_batch = next(en_fr_data_iterator)

first_input_batch_sorted, first_input_batch, first_target_batch, input_lengths, original_idx = first_batch

In [169]:
print('Input batch:')
print(first_input_batch)
print('Input batch sorted:')
print(first_input_batch_sorted)

Input batch:
tensor([[ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
        [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
        [ 6,  7,  7,  8,  8,  9, 10, 10, 12, 13],
        [ 0,  6,  6,  6,  6,  6, 11, 11,  6,  6]])
Input batch sorted:
tensor([[ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
        [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
        [ 7,  7,  8,  8,  9, 10, 10, 12, 13,  6],
        [ 6,  6,  6,  6,  6, 11, 11,  6,  6,  0]])


In [164]:
max_length = input_lengths[0].item()

first_input_batch_sorted.gather(dim=1, index=argsort_idx.repeat(max_length, 1))

tensor([[ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
        [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
        [ 6,  7,  7,  8,  8,  9, 10, 10, 12, 13],
        [ 0,  6,  6,  6,  6,  6, 11, 11,  6,  6]])

And that's it! We now have a dataloader class which can pad input and target batches.

In [ ]:
en_embedding = nn.Embedding(num_embeddings=len(en_fr_dataset.input_vocab), embedding_dim=3, padding_idx=0)

In [ ]:
embedded_input = en_embedding(first_input_batch)
print(embedded_input.size())